In [3]:
import mysql.connector
# Connect to the MySQL server
connection = mysql.connector.connect(
    host="13.235.95.5",
    user="root",
    password="tanuj1221",
    database="demo1"
)

if connection.is_connected():
    print("Connected to MySQL database")

    # Create a cursor object
    cursor = connection.cursor()

    # Define the SQL query to drop the table
    table_name = "sa"
    drop_table_query = f"DROP TABLE IF EXISTS {table_name}"

    # Execute the SQL query
    cursor.execute(drop_table_query)

    # Commit the changes
    connection.commit()

    # Close the cursor and the connection
    cursor.close()
    connection.close()
    print("Table deleted successfully")
else:
    print("Failed to connect to MySQL database")


Connected to MySQL database
Table deleted successfully


In [3]:
import mysql.connector

# Connect to the MySQL server
connection = mysql.connector.connect(
    host="13.235.95.5",
    user="root",
    password="tanuj1221",
    database="demo1"
)

# Create a cursor object using the connection
cursor = connection.cursor()

# Query to select all records from the institutedb table
query = "SELECT * FROM student14"

try:
    # Execute the query
    cursor.execute(query)

    # Fetch all the rows
    results = cursor.fetchall()

    # Check if any rows are returned
    if results:
        for row in results:
            print(row)  # Print each row
    else:
        print("No data found in the table.")

except mysql.connector.Error as e:
    print(f"Error: {e}")

finally:
    # Close the cursor and connection to free up the resources
    cursor.close()
    connection.close()


('12343', '1221', '13124', '1', '21/2/2024', '29/2/2024', 'Rajesh', 'Mishra', 'Shanti', 'Murali', '[101]', 'paid', '[11]', '2024', 'yes', '67', 'no', '1', '', '', '')
('74644', '2561', '13124', None, '21/2/2024', '29/2/2024', 'Tanuj', 'Chavan', 'pooja', 'vijay', '[101]', 'paid', '[11]', '2024', 'yes', '89', 'no', '1', '...', '9786756453', 'tcchavan999@gmail.com')


In [ ]:

import Levenshtein
import re   
from collections import Counter
import string

def similarity(word1, word2):
    similarity = 1 - Levenshtein.distance(word1, word2) / max(len(word1), len(word2))
    return similarity

def helper(word1, word2):
    if word1 == word2:
        return -1
    elif word1 != word2 and word1.lower() == word2.lower():
        return 1
    elif word1 != word2 and similarity(word1, word2) >= 0.5:
        return 0
    else:
        return 2

mistakes = {'spelling': [], 'missing': [], 'extra': [], 'case': [], 'paramistake': []}

def has_punctuation(word):
    return any(char in string.punctuation for char in word)

def count_mistakes(original, answer):
    global mistakes
    mistakes = {
        'spelling': [],
        'missing': [],
        'extra': [],
        'case': [],
        'paramistake': []
    }

    paramis = original.count("\n") - answer.count("\n")
    original = original.replace("\n", " ")
    answer = answer.replace("\n", " ")
    for i in range(abs(paramis)):
        mistakes['paramistake'].append('a')

    original_words = original.split()
    answer_words = answer.split()

    original_words = [word for word in original_words if not has_punctuation(word)]
    answer_words = [word for word in answer_words if not has_punctuation(word)]

    i = 0
    j = 0

    while i < len(original_words) and j < len(answer_words):
        if j < len(answer_words) - 1:
            xd = j + 1
            vd = i + 1
        else:
            xd = j
            vd = i

        if helper(original_words[i], answer_words[j]) == -1:   
            i += 1
            j += 1
        elif helper(original_words[i], answer_words[j]) == 0 and helper(original_words[i], answer_words[xd]) in [0, 2]:
            mistakes['spelling'].append(answer_words[j])
            i += 1
            j += 1
        elif helper(original_words[i], answer_words[j]) == 0 and helper(original_words[i], answer_words[xd]) not in [0, 2]:
            mistakes['extra'].append(answer_words[j])
            j += 1
        elif helper(original_words[i], answer_words[j]) == 1:
            mistakes['case'].append(answer_words[j])
            i += 1
            j += 1
        elif helper(original_words[i], answer_words[j]) == 2:
            found_match = False
            
            if answer_words[j] not in original_words[i:i+3] or answer_words[j] in original_words[i:i+3]:
                sh = []
                for k in range(i, min(i + 3, len(original_words))):
                    if helper(original_words[k], answer_words[j]) in [-1, 0, 1]:
                        found_match = True
                        sh = original_words[i:k]
                        for word in sh:
                            mistakes['missing'].append(word)
                        i = k
                        break
                    k += 1

            if not found_match and original_words[i] not in answer_words[j:j+3] or original_words[i] in answer_words[j:j+3]:
                for h in range(j, min(j + 3, len(answer_words))):
                    if helper(original_words[i], answer_words[h]) in [-1, 0, 1]:
                        found_match = True
                        li = answer_words[j:h]
                        for wor in li:
                            mistakes['extra'].append(wor)
                        j = h
                        break
                    h += 1
            
            if not found_match:              
                for k in range(j, len(answer_words)):
                    if helper(original_words[i], answer_words[k]) in [-1, 0, 1]:
                        found_match = True
                        ex = answer_words[j:k]
                        curr_index = j
                        for e in ex:
                            mistakes['extra'].append(e)
                            curr_index += 1
                        j = k 
                        break
                    k += 1
                if not found_match:
                    for k in range(i, len(original_words)):
                        if helper(original_words[k], answer_words[j]) in [-1, 0, 1]:
                            found_match = True
                            missW = original_words[i:k]
                            for m in missW:
                                mistakes['missing'].append(m)
                            i = k
                            break
                        k += 1
                
                if not found_match:
                    mistakes['extra'].append(answer_words[j])
                    mistakes['missing'].append(original_words[i])
                    i += 1
                    j += 1

    if i < len(original_words):
        missedWords = original_words[i:]
        for w in missedWords:
            mistakes['missing'].append(w)

    if j < len(answer_words):
        extraWords = answer_words[j:]
        curr_index = j
        for w in extraWords:
            mistakes['extra'].append(w)
            curr_index += 1

    print(mistakes)       

    return mistakes

In [4]:
import Levenshtein
import string

def similarity(word1, word2):
    return 1 - Levenshtein.distance(word1, word2) / max(len(word1), len(word2))

def helper(word1, word2):
    if word1 == word2:
        return -1
    elif word1.lower() == word2.lower():
        return 1
    elif similarity(word1, word2) >= 0.5:
        return 0
    else:
        return 2

def has_punctuation(word):
    return any(char in string.punctuation for char in word)

def count_mistakes(original, answer, ignore_list=[]):
    mistakes = {
        'spelling': [],
        'missing': [],
        'extra': [],
        'case': [],
        'paramistake': []
    }

    paramis = original.count("\n") - answer.count("\n")
    original = original.replace("\n", " ")
    answer = answer.replace("\n", " ")
    for _ in range(abs(paramis)):
        mistakes['paramistake'].append('a')

    original_words = [word for word in original.split() if not has_punctuation(word)]
    answer_words = [word for word in answer.split() if not has_punctuation(word)]

    i = 0
    j = 0

    while i < len(original_words) and j < len(answer_words):
        current_error_type = helper(original_words[i], answer_words[j])
        if current_error_type == -1:  # No error
            i += 1
            j += 1
        elif current_error_type == 0:  # Spelling mistake
            if answer_words[j] not in ignore_list:
                mistakes['spelling'].append((answer_words[j], j))
            i += 1
            j += 1
        elif current_error_type == 1:  # Case error
            if answer_words[j] not in ignore_list:
                mistakes['case'].append((answer_words[j], j))
            i += 1
            j += 1
        else:  # Possible extra or missing words
            original_future = original_words[i:i+3]
            answer_future = answer_words[j:j+3]
            # Find matching index if any
            found = False
            for index in range(3):
                if i + index < len(original_words) and helper(original_words[i + index], answer_words[j]) in [-1, 0, 1]:
                    found = True
                    if i + index != i:
                        for missing_word in original_words[i:i + index]:
                            if missing_word not in ignore_list:
                                mistakes['missing'].append((missing_word, i))
                    i = i + index
                    break
            if not found:
                if answer_words[j] not in ignore_list:
                    mistakes['extra'].append((answer_words[j], j))
                j += 1

    if i < len(original_words):
        for index, w in enumerate(original_words[i:], i):
            if w not in ignore_list:
                mistakes['missing'].append((w, index))

    if j < len(answer_words):
        for index, w in enumerate(answer_words[j:], j):
            if w not in ignore_list:
                mistakes['extra'].append((w, index))

    return mistakes




ignore_words = ['//१//', '//२//','//३//','//४//']  # Add your words to ignore here
original_text = """
सभापति महोदय,

इस सदन के सामने जो विधेयक अब आया है उसका समर्थन करने के लिये मैं खड़ा हूँ। सिनेमा उद्योग हमारे देश का बहुत पुराना उद्योग है। दिनों / दिन फिल्में बनाना महंगा होता जा रहा है। फिल्म  बढ़ाने में करोड़ों रुपया लगता है। फिल्मों में काम करनेवाले जो बड़े कलाकार होते हैं वे तो अच्छी तरह से // रह सकते हैं। लेकिन जो छोटे छोटे कलाकार है वे बहुत गरीबी की हालत में रहते हैं। उनको कुछ राहत देने के लिए माननीय मंत्री महोदयजी ने इस विधेयक /// के द्वारा प्रयत्न किया है। इसलिये माननीय मंत्री महोदयजी धन्यवाद के पात्र है। देश की जनता पर अच्छी फिल्मों का अच्छा और गलत फिल्मों का बुरा असर पड़ता है । //१// इस विधेयक से निश्चय ही अच्छा असर पड़ेगा और अच्छी फिल्में बनेगी। इनके साथ साथ सर्कस आदि में भी जो लोग काम करते हैं उनके लिये भी ऐसा / ही प्रबंध अब किया जा रहा है यह बड़ी सराहनीय बात है। सर्कस में जो काम करते है, वे भी अच्छे कलाकार होते है। सर्कस में तो जानवर भी काम // करते है। वहाँ हाथी, शेर, घोडे आदि बहुत से जानवर होते है। अगर कोई सर्कस कंपनी अयशस्वी हो जाती थी तो बड़ी गरीबी की हालत में इन में से बहुत से /// कलाकारों को अपना जीवन बीताना पड़ता था। उनको कहीं पैर रखने के लिए भी जगह नहीं मिल पाती थी। इसलिये उनके लिये भी अब कुछ न कुछ प्रबंध किया //२// जा रहा है यह बड़ी अच्छी बात है। गावों में छोटे छोटे कलाकार अपनी कंपनी बनाकर लोगों का मनोरंजन करने का काम अच्छी तरह से करते हैं। लेकिन हर / साल कुछ महिनों में ही उनको काम मिलता है। इन लोगों को अपनी प्रतिभा का विकास करने के लिये शक्ति मिलें इसलिए अब प्रबंध किया जा रहा है यह बहुत // अच्छी बात है। हमारे देश से कई फिल्में विदेशों में भी जाती हैं और वे हमारी प्रगति वहां दिखलाती हैं। मैं समझता हूँ कि करीब करीब सौ से ज्यादा देशों /// में हमारी फिल्मे निर्यात होती हैं और विश्व के कोने में इन फिल्मों की लोकप्रियता बढ़ रही है। इसलिये और भी आवश्यक है कि अच्छी से अच्छी फिल्में हमारे //३// यहां बने ताकि उनसे हमें अच्छा लाभ प्राप्त हो । माननीय मंत्री महोदय आज जो विधेयक यहां लाये है वह बहुत ही अच्छा है। खास कर गरीब छोटे छोटे सिने कलाकारो / के लिये अब एक कल्याणकारी निधी खड़ा किया जायेगा । इस उद्योग में जिनको बहुत कम पगार मिलता है उनको कुछ अधिक सहायता दी जायेगी । मैं अब. सदन का ज्यादा // वक्त न लेते हुए इस विधेयक का समर्थन करता हूँ और माननीय मंत्री महोदय को धन्यवाद देता हूँ । माननीय मंत्री महोदय को फिल्मों के बारे में अच्छा तर्जुबा है । इसलिये /// उनको भी धन्यवाद और सभापति महोदयजी, आपने मुझे मेरे ये विचार व्यक्त करने के लिये अवसर दिया उसके लिये आपको भी धन्यवाद देते हुए मैं अपना भाषण समाप्त करता हूँ । //४//
"""
answer_text = """
सभापति महोदय,

इस सदन के सामने जो विधेयक अब आया है उसका समर्थन करने के लिये मैं खड़ा हूँ। सिनेमा उद्योग हमारे देश का बहुत पुराना उद्योग है। दिनों / दिन फिल्में बनाना महंगा होता जा रहा है। फिल्म  बढ़ाने में करोड़ों रुपया लगता है। फिल्मों में काम करनेवाले जो बड़े कलाकार होते हैं वे तो अच्छी तरह से // रह सकते हैं। लेकिन जो छोटे छोटे कलाकार है वे बहुत गरीबी की हालत में रहते हैं। उनको कुछ राहत देने के लिए माननीय मंत्री महोदयजी ने इस विधेयक /// के द्वारा प्रयत्न किया है। इसलिये माननीय मंत्री महोदयजी धन्यवाद के पात्र है। देश की जनता पर अच्छी फिल्मों का अच्छा और गलत फिल्मों का बुरा असर पड़ता है । //१// इस विधेयक से निश्चय ही अच्छा असर पड़ेगा और अच्छी फिल्में बनेगी। इनके साथ साथ सर्कस आदि में भी जो लोग काम करते हैं उनके लिये भी ऐसा / ही प्रबंध अब किया जा रहा है यह बड़ी सराहनीय बात है। सर्कस में जो काम करते है, वे भी अच्छे कलाकार होते है। सर्कस में तो जानवर भी काम // करते है। वहाँ हाथी, शेर, घोडे आदि बहुत से जानवर होते है। अगर कोई सर्कस कंपनी अयशस्वी हो जाती थी तो बड़ी गरीबी की हालत में इन में से बहुत से /// कलाकारों को अपना जीवन बीताना पड़ता था। उनको कहीं पैर रखने के लिए भी जगह नहीं मिल पाती थी। इसलिये उनके लिये भी अब कुछ न कुछ प्रबंध किया //२// जा रहा है यह बड़ी अच्छी बात है। गावों में छोटे छोटे कलाकार अपनी कंपनी बनाकर लोगों का मनोरंजन करने का काम अच्छी तरह से करते हैं। लेकिन हर / साल कुछ महिनों में ही उनको काम मिलता है। इन लोगों को अपनी प्रतिभा का विकास करने के लिये शक्ति मिलें इसलिए अब प्रबंध किया जा रहा है यह बहुत // अच्छी बात है। हमारे देश से कई फिल्में विदेशों में भी जाती हैं और वे हमारी प्रगति वहां दिखलाती हैं। मैं समझता हूँ कि करीब करीब सौ से ज्यादा देशों /// में हमारी फिल्मे निर्यात होती हैं और विश्व के कोने में इन फिल्मों की लोकप्रियता बढ़ रही है। इसलिये और भी आवश्यक है कि अच्छी से अच्छी फिल्में हमारे //३// यहां बने ताकि उनसे हमें अच्छा लाभ प्राप्त हो । माननीय मंत्री महोदय आज जो विधेयक यहां लाये है वह बहुत ही अच्छा है। खास कर गरीब छोटे छोटे सिने कलाकारो / के लिये अब एक कल्याणकारी निधी खड़ा किया जायेगा । इस उद्योग में जिनको बहुत कम पगार मिलता है उनको कुछ अधिक सहायता दी जायेगी । मैं अब. सदन का ज्यादा // वक्त न लेते हुए इस विधेयक का समर्थन करता हूँ और माननीय मंत्री महोदय को धन्यवाद देता हूँ । माननीय मंत्री महोदय को फिल्मों के बारे में अच्छा तर्जुबा है । इसलिये /// उनको भी धन्यवाद और सभापति महोदयजी, आपने मुझ मेरे ये विचार व्यक्त करने के लिये अवसर दिया उसके लिये आपको भी धन्यवाद देते हुए मैं अपना भाषण समाप्त करता हूँ । //४//
"""
count_mistakes(original_text, answer_text, ignore_words)


{'spelling': [('मुझ', 448)],
 'missing': [],
 'extra': [],
 'case': [],
 'paramistake': []}

In [5]:
def similarity(word1, word2):
    return 1 - Levenshtein.distance(word1, word2) / max(len(word1), len(word2))

def helper(word1, word2):
    if word1 == word2:
        return -1
    elif word1.lower() == word2.lower():
        return 1
    elif similarity(word1, word2) >= 0.5:
        return 0
    else:
        return 2

def has_punctuation(word):
    return any(char in string.punctuation for char in word)

def count_mistakes(original, answer, ignore_list=[]):
    mistakes = {
        'spelling': [],
        'missing': [],
        'extra': [],
        'case': [],
        'paramistake': []
    }

    paramis = original.count("\n") - answer.count("\n")
    original = original.replace("\n", " ")
    answer = answer.replace("\n", " ")
    for _ in range(abs(paramis)):
        mistakes['paramistake'].append('a')

    original_words = [word for word in original.split() if not has_punctuation(word)]
    answer_words = [word for word in answer.split() if not has_punctuation(word)]

    i = 0
    j = 0

    while i < len(original_words) and j < len(answer_words):
        current_error_type = helper(original_words[i], answer_words[j])
        if current_error_type == -1:  # No error
            i += 1
            j += 1
        elif current_error_type == 0:  # Spelling mistake
            if answer_words[j] not in ignore_list:
                mistakes['spelling'].append(answer_words[j])
            i += 1
            j += 1
        elif current_error_type == 1:  # Case error
            if answer_words[j] not in ignore_list:
                mistakes['case'].append(answer_words[j])
            i += 1
            j += 1
        else:  # Possible extra or missing words
            original_future = original_words[i:i+3]
            answer_future = answer_words[j:j+3]
            found = False
            for index in range(3):
                if i + index < len(original_words) and helper(original_words[i + index], answer_words[j]) in [-1, 0, 1]:
                    found = True
                    if i + index != i:
                        mistakes['missing'].extend(original_words[i:i + index])
                    i = i + index
                    break
            if not found:
                if answer_words[j] not in ignore_list:
                    mistakes['extra'].append(answer_words[j])
                j += 1

    if i < len(original_words):
        mistakes['missing'].extend([w for w in original_words[i:] if w not in ignore_list])

    if j < len(answer_words):
        mistakes['extra'].extend([w for w in answer_words[j:] if w not in ignore_list])

    return mistakes

ignore_words = ['//१//', '//२//','//३//','//४//']  # Add your words to ignore here
original_text = """
सभापति महोदय,

इस सदन के सामने जो विधेयक अब आया है उसका समर्थन करने के लिये मैं खड़ा हूँ। सिनेमा उद्योग हमारे देश का बहुत पुराना उद्योग है। दिनों / दिन फिल्में बनाना महंगा होता जा रहा है। फिल्म  बढ़ाने में करोड़ों रुपया लगता है। फिल्मों में काम करनेवाले जो बड़े कलाकार होते हैं वे तो अच्छी तरह से // रह सकते हैं। लेकिन जो छोटे छोटे कलाकार है वे बहुत गरीबी की हालत में रहते हैं। उनको कुछ राहत देने के लिए माननीय मंत्री महोदयजी ने इस विधेयक /// के द्वारा प्रयत्न किया है। इसलिये माननीय मंत्री महोदयजी धन्यवाद के पात्र है। देश की जनता पर अच्छी फिल्मों का अच्छा और गलत फिल्मों का बुरा असर पड़ता है । //१// इस विधेयक से निश्चय ही अच्छा असर पड़ेगा और अच्छी फिल्में बनेगी। इनके साथ साथ सर्कस आदि में भी जो लोग काम करते हैं उनके लिये भी ऐसा / ही प्रबंध अब किया जा रहा है यह बड़ी सराहनीय बात है। सर्कस में जो काम करते है, वे भी अच्छे कलाकार होते है। सर्कस में तो जानवर भी काम // करते है। वहाँ हाथी, शेर, घोडे आदि बहुत से जानवर होते है। अगर कोई सर्कस कंपनी अयशस्वी हो जाती थी तो बड़ी गरीबी की हालत में इन में से बहुत से /// कलाकारों को अपना जीवन बीताना पड़ता था। उनको कहीं पैर रखने के लिए भी जगह नहीं मिल पाती थी। इसलिये उनके लिये भी अब कुछ न कुछ प्रबंध किया //२// जा रहा है यह बड़ी अच्छी बात है। गावों में छोटे छोटे कलाकार अपनी कंपनी बनाकर लोगों का मनोरंजन करने का काम अच्छी तरह से करते हैं। लेकिन हर / साल कुछ महिनों में ही उनको काम मिलता है। इन लोगों को अपनी प्रतिभा का विकास करने के लिये शक्ति मिलें इसलिए अब प्रबंध किया जा रहा है यह बहुत // अच्छी बात है। हमारे देश से कई फिल्में विदेशों में भी जाती हैं और वे हमारी प्रगति वहां दिखलाती हैं। मैं समझता हूँ कि करीब करीब सौ से ज्यादा देशों /// में हमारी फिल्मे निर्यात होती हैं और विश्व के कोने में इन फिल्मों की लोकप्रियता बढ़ रही है। इसलिये और भी आवश्यक है कि अच्छी से अच्छी फिल्में हमारे //३// यहां बने ताकि उनसे हमें अच्छा लाभ प्राप्त हो । माननीय मंत्री महोदय आज जो विधेयक यहां लाये है वह बहुत ही अच्छा है। खास कर गरीब छोटे छोटे सिने कलाकारो / के लिये अब एक कल्याणकारी निधी खड़ा किया जायेगा । इस उद्योग में जिनको बहुत कम पगार मिलता है उनको कुछ अधिक सहायता दी जायेगी । मैं अब. सदन का ज्यादा // वक्त न लेते हुए इस विधेयक का समर्थन करता हूँ और माननीय मंत्री महोदय को धन्यवाद देता हूँ । माननीय मंत्री महोदय को फिल्मों के बारे में अच्छा तर्जुबा है । इसलिये /// उनको भी धन्यवाद और सभापति महोदयजी, आपने मुझे मेरे ये विचार व्यक्त करने के लिये अवसर दिया उसके लिये आपको भी धन्यवाद देते हुए मैं अपना भाषण समाप्त करता हूँ । //४//
"""
answer_text = """
सभापति महोदय,

इस सदन के सामने जो विधेयक अब आया है उसका समर्थन करने के लिये मैं खड़ा हूँ। सिनेमा उद्योग हमारे देश का बहुत पुराना उद्योग है। दिनों / दिन फिल्में बनाना महंगा होता जा रहा है। फिल्म  बढ़ाने में करोड़ों रुपया लगता है। फिल्मों में काम करनेवाले जो बड़े कलाकार होते हैं वे तो अच्छी तरह से // रह सकते हैं। लेकिन जो छोटे छोटे कलाकार है वे बहुत गरीबी की हालत में रहते हैं। उनको कुछ राहत देने के लिए माननीय मंत्री महोदयजी ने इस विधेयक /// के द्वारा प्रयत्न किया है। इसलिये माननीय मंत्री महोदयजी धन्यवाद के पात्र है। देश की जनता पर अच्छी फिल्मों का अच्छा और गलत फिल्मों का बुरा असर पड़ता है । //१// इस विधेयक से निश्चय ही अच्छा असर पड़ेगा और अच्छी फिल्में बनेगी। इनके साथ साथ सर्कस आदि में भी जो लोग काम करते हैं उनके लिये भी ऐसा / ही प्रबंध अब किया जा रहा है यह बड़ी सराहनीय बात है। सर्कस में जो काम करते है, वे भी अच्छे कलाकार होते है। सर्कस में तो जानवर भी काम // करते है। वहाँ हाथी, शेर, घोडे आदि बहुत से जानवर होते है। अगर कोई सर्कस कंपनी अयशस्वी हो जाती थी तो बड़ी गरीबी की हालत में इन में से बहुत से /// कलाकारों को अपना जीवन बीताना पड़ता था। उनको कहीं पैर रखने के लिए भी जगह नहीं मिल पाती थी। इसलिये उनके लिये भी अब कुछ न कुछ प्रबंध किया //२// जा रहा है यह बड़ी अच्छी बात है। गावों में छोटे छोटे कलाकार अपनी कंपनी बनाकर लोगों का मनोरंजन करने का काम अच्छी तरह से करते हैं। लेकिन हर / साल कुछ महिनों में ही उनको काम मिलता है। इन लोगों को अपनी प्रतिभा का विकास करने के लिये शक्ति मिलें इसलिए अब प्रबंध किया जा रहा है यह बहुत // अच्छी बात है। हमारे देश से कई फिल्में विदेशों में भी जाती हैं और वे हमारी प्रगति वहां दिखलाती हैं। मैं समझता हूँ कि करीब करीब सौ से ज्यादा देशों /// में हमारी फिल्मे निर्यात होती हैं और विश्व के कोने में इन फिल्मों की लोकप्रियता बढ़ रही है। इसलिये और भी आवश्यक है कि अच्छी से अच्छी फिल्में हमारे //३// यहां बने ताकि उनसे हमें अच्छा लाभ प्राप्त हो । माननीय मंत्री महोदय आज जो विधेयक यहां लाये है वह बहुत ही अच्छा है। खास कर गरीब छोटे छोटे सिने कलाकारो / के लिये अब एक कल्याणकारी निधी खड़ा किया जायेगा । इस उद्योग में जिनको बहुत कम पगार मिलता है उनको कुछ अधिक सहायता दी जायेगी । मैं अब. सदन का ज्यादा // वक्त न लेते हुए इस विधेयक का समर्थन करता हूँ और माननीय मंत्री महोदय को धन्यवाद देता हूँ । माननीय मंत्री महोदय को फिल्मों के बारे में अच्छा तर्जुबा है । इसलिये /// उनको भी धन्यवाद और सभापति महोदयजी, आपने मुझ मेरे ये विचार व्यक्त करने के लिये अवसर दिया उसके लिये आपको भी धन्यवाद देते हुए मैं अपना भाषण समाप्त करता हूँ । //४//
"""
count_mistakes(original_text, answer_text, ignore_words)

'्'

In [9]:
import string
from Levenshtein import distance as levenshtein_distance

def similarity(word1, word2):
    return 1 - levenshtein_distance(word1, word2) / max(len(word1), len(word2))

def helper(word1, word2):
    if word1 == word2:
        return -1
    elif word1.lower() == word2.lower():
        return 1
    elif similarity(word1, word2) >= 0.5:
        return 0
    else:
        return 2

def has_punctuation(word):
    return any(char in string.punctuation for char in word)

def count_mistakes_and_rebuild_passage(original, answer, ignore_list=[]):
    mistakes = {
        'spelling': [],
        'missing': [],
        'extra': [],
        'case': [],
        'paramistake': []
    }
    rebuilt_answer = []
    
    paramis = original.count("\n") - answer.count("\n")
    original = original.replace("\n", " ")
    answer = answer.replace("\n", " ")
    for _ in range(abs(paramis)):
        mistakes['paramistake'].append('a')

    original_words = original.split()
    answer_words = answer.split()

    i = 0
    j = 0

    while i < len(original_words) and j < len(answer_words):
        original_word = original_words[i]
        answer_word = answer_words[j]

        if has_punctuation(original_word) or has_punctuation(answer_word):
            rebuilt_answer.append(answer_word)
            j += 1
            i += 1
            continue
        
        current_error_type = helper(original_word, answer_word)
        if current_error_type == -1:  # No error
            rebuilt_answer.append(answer_word)
            i += 1
            j += 1
        elif current_error_type == 0:  # Spelling mistake
            if answer_word not in ignore_list:
                mistakes['spelling'].append(answer_word)
                rebuilt_answer.append(f"[{original_word}->{answer_word}]")
            i += 1
            j += 1
        elif current_error_type == 1:  # Case error
            if answer_word not in ignore_list:
                mistakes['case'].append(answer_word)
                rebuilt_answer.append(f"[{original_word}->{answer_word}]")
            i += 1
            j += 1
        else:  # Possible extra or missing words
            original_future = original_words[i:i+3]
            answer_future = answer_words[j:j+3]
            found = False
            for index in range(3):
                if i + index < len(original_words) and helper(original_words[i + index], answer_words[j]) in [-1, 0, 1]:
                    found = True
                    if i + index != i:
                        mistakes['missing'].extend(original_words[i:i + index])
                        rebuilt_answer.append(f"{' '.join(original_words[i:i+index])}")
                    i += index
                    break
            if not found:
                if answer_word not in ignore_list:
                    mistakes['extra'].append(answer_word)
                    rebuilt_answer.append(f"[->{answer_word}]")
                j += 1

    if i < len(original_words):
        mistakes['missing'].extend(original_words[i:])
        rebuilt_answer.append(f"{' '.join(original_words[i:])}")

    if j < len(answer_words):
        mistakes['extra'].extend(answer_words[j:])
        rebuilt_answer.append(f"{' '.join(['->' + w for w in answer_words[j:] if w not in ignore_list])}")

    return mistakes, " ".join(rebuilt_answer)


ignore_words = ['//१//', '//२//','//३//','//४//']  # Add your words to ignore here
original_text = """
सभापति महोदय,

इस सदन के सामने जो विधेयक अब आया है उसका समर्थन करने के लिये मैं खड़ा हूँ। सिनेमा उद्योग हमारे देश का बहुत पुराना उद्योग है। दिनों / दिन फिल्में बनाना महंगा होता जा रहा है। फिल्म  बढ़ाने में करोड़ों रुपया लगता है। फिल्मों में काम करनेवाले जो बड़े कलाकार होते हैं वे तो अच्छी तरह से // रह सकते हैं। लेकिन जो छोटे छोटे कलाकार है वे बहुत गरीबी की हालत में रहते हैं। उनको कुछ राहत देने के लिए माननीय मंत्री महोदयजी ने इस विधेयक /// के द्वारा प्रयत्न किया है। इसलिये माननीय मंत्री महोदयजी धन्यवाद के पात्र है। देश की जनता पर अच्छी फिल्मों का अच्छा और गलत फिल्मों का बुरा असर पड़ता है । //१// इस विधेयक से निश्चय ही अच्छा असर पड़ेगा और अच्छी फिल्में बनेगी। इनके साथ साथ सर्कस आदि में भी जो लोग काम करते हैं उनके लिये भी ऐसा / ही प्रबंध अब किया जा रहा है यह बड़ी सराहनीय बात है। सर्कस में जो काम करते है, वे भी अच्छे कलाकार होते है। सर्कस में तो जानवर भी काम // करते है। वहाँ हाथी, शेर, घोडे आदि बहुत से जानवर होते है। अगर कोई सर्कस कंपनी अयशस्वी हो जाती थी तो बड़ी गरीबी की हालत में इन में से बहुत से /// कलाकारों को अपना जीवन बीताना पड़ता था। उनको कहीं पैर रखने के लिए भी जगह नहीं मिल पाती थी। इसलिये उनके लिये भी अब कुछ न कुछ प्रबंध किया //२// जा रहा है यह बड़ी अच्छी बात है। गावों में छोटे छोटे कलाकार अपनी कंपनी बनाकर लोगों का मनोरंजन करने का काम अच्छी तरह से करते हैं। लेकिन हर / साल कुछ महिनों में ही उनको काम मिलता है। इन लोगों को अपनी प्रतिभा का विकास करने के लिये शक्ति मिलें इसलिए अब प्रबंध किया जा रहा है यह बहुत // अच्छी बात है। हमारे देश से कई फिल्में विदेशों में भी जाती हैं और वे हमारी प्रगति वहां दिखलाती हैं। मैं समझता हूँ कि करीब करीब सौ से ज्यादा देशों /// में हमारी फिल्मे निर्यात होती हैं और विश्व के कोने में इन फिल्मों की लोकप्रियता बढ़ रही है। इसलिये और भी आवश्यक है कि अच्छी से अच्छी फिल्में हमारे //३// यहां बने ताकि उनसे हमें अच्छा लाभ प्राप्त हो । माननीय मंत्री महोदय आज जो विधेयक यहां लाये है वह बहुत ही अच्छा है। खास कर गरीब छोटे छोटे सिने कलाकारो / के लिये अब एक कल्याणकारी निधी खड़ा किया जायेगा । इस उद्योग में जिनको बहुत कम पगार मिलता है उनको कुछ अधिक सहायता दी जायेगी । मैं अब. सदन का ज्यादा // वक्त न लेते हुए इस विधेयक का समर्थन करता हूँ और माननीय मंत्री महोदय को धन्यवाद देता हूँ । माननीय मंत्री महोदय को फिल्मों के बारे में अच्छा तर्जुबा है । इसलिये /// उनको भी धन्यवाद और सभापति महोदयजी, आपने मुझे मेरे ये विचार व्यक्त करने के लिये अवसर दिया उसके लिये आपको भी धन्यवाद देते हुए मैं अपना भाषण समाप्त करता हूँ । //४//
"""
answer_text = """
सभापति महोदय,

इस सदन के सामने जो विधेयक अब आया है उसका समर्थन करने के लिये मैं खड़ा हूँ। सिनेमा उद्योग हमारे देश का बहुत पुराना उद्योग है। दिनों / दिन फिल्में बनाना महंगा होता जा रहा है। फिल्म  बढ़ान में करोड़ों रुपया लगता है। फिल्मों में काम करनेवाले जो बड़े कलाकार होते हैं वे तो अच्छी तरह से // रह सकते हैं। लेकिन जो छोटे छोटे कलाकार है वे बहुत गरीबी की हालत में रहते हैं। उनको कुछ राहत देने के लिए माननीय मंत्री महोदयजी ने इस विधेयक /// के द्वारा प्रयत्न किया है। इसलिये माननीय मंत्री महोदयजी धन्यवाद के पात्र है। देश की जनता पर अच्छी फिल्मों का अच्छा और गलत फिल्मों का बुरा असर पड़ता है । //१// इस विधेयक से निश्चय ही अच्छा असर पड़ेगा और अच्छी फिल्में बनेगी। इनके साथ साथ सर्कस आदि में भी जो लोग काम करते हैं उनके लिये भी ऐसा / ही प्रबंध अब किया जा रहा है यह बड़ी सराहनीय बात है। सर्कस में जो काम करते है, वे भी अच्छे कलाकार होते है। सर्कस में तो जानवर भी काम // करते है। वहाँ हाथी, शेर, घोडे आदि बहुत से जानवर होते है। अगर कोई सर्कस कंपनी अयशस्वी हो जाती थी तो बड़ी गरीबी की हालत में इन में से बहुत से /// कलाकारों को अपना जीवन बीताना पड़ता था। उनको कहीं पैर रखने के लिए भी जगह नहीं मिल पाती थी। इसलिये उनके लिये भी अब कुछ न कुछ प्रबंध किया //२// जा रहा है यह बड़ी अच्छी बात है। गावों में छोटे छोटे कलाकार अपनी कंपनी बनाकर लोगों का मनोरंजन करने का काम अच्छी तरह से करते हैं। लेकिन हर / साल कुछ महिनों में ही उनको काम मिलता है। इन लोगों को अपनी प्रतिभा का विकास करने के लिये शक्ति मिलें इसलिए अब प्रबंध किया जा रहा है यह बहुत // अच्छी बात है। हमारे देश से कई फिल्में विदेशों में भी जाती हैं और वे हमारी प्रगति वहां दिखलाती हैं। मैं समझता हूँ कि करीब करीब सौ से ज्यादा देशों /// में हमारी फिल्मे निर्यात होती हैं और विश्व के कोने में इन फिल्मों की लोकप्रियता बढ़ रही है। इसलिये और भी आवश्यक है कि अच्छी से अच्छी फिल्में हमारे //३// यहां बने ताकि उनसे हमें अच्छा लाभ प्राप्त हो । माननीय मंत्री महोदय आज जो विधेयक यहां लाये है वह बहुत ही अच्छा है। खास कर गरीब छोटे छोटे सिने कलाकारो / के लिये अब एक कल्याणकारी निधी खड़ा किया जायेगा । इस उद्योग में जिनको बहुत कम पगार मिलता है उनको कुछ अधिक सहायता दी जायेगी । मैं अब. सदन का ज्यादा // वक्त न लेते हुए इस विधेयक का समर्थन करता हूँ और माननीय मंत्री महोदय को धन्यवाद देता हूँ । माननीय मंत्री महोदय को फिल्मों के बारे में अच्छा तर्जुबा है । इसलिये /// उनको भी धन्यवाद और सभापति महोदयजी, आपने मुझ मेरे ये विचार व्यक्त करने के लिये अवसर दिया उसके लिये आपको भी धन्यवाद देते हुए मैं अपना भाषण समाप्त करता हूँ । //४//
"""
count_mistakes_and_rebuild_passage(original_text, answer_text, ignore_words)

({'spelling': ['बढ़ान', 'मुझ'],
  'missing': [],
  'extra': [],
  'case': [],
  'paramistake': []},
 'सभापति महोदय, इस सदन के सामने जो विधेयक अब आया है उसका समर्थन करने के लिये मैं खड़ा हूँ। सिनेमा उद्योग हमारे देश का बहुत पुराना उद्योग है। दिनों / दिन फिल्में बनाना महंगा होता जा रहा है। फिल्म [बढ़ाने->बढ़ान] में करोड़ों रुपया लगता है। फिल्मों में काम करनेवाले जो बड़े कलाकार होते हैं वे तो अच्छी तरह से // रह सकते हैं। लेकिन जो छोटे छोटे कलाकार है वे बहुत गरीबी की हालत में रहते हैं। उनको कुछ राहत देने के लिए माननीय मंत्री महोदयजी ने इस विधेयक /// के द्वारा प्रयत्न किया है। इसलिये माननीय मंत्री महोदयजी धन्यवाद के पात्र है। देश की जनता पर अच्छी फिल्मों का अच्छा और गलत फिल्मों का बुरा असर पड़ता है । //१// इस विधेयक से निश्चय ही अच्छा असर पड़ेगा और अच्छी फिल्में बनेगी। इनके साथ साथ सर्कस आदि में भी जो लोग काम करते हैं उनके लिये भी ऐसा / ही प्रबंध अब किया जा रहा है यह बड़ी सराहनीय बात है। सर्कस में जो काम करते है, वे भी अच्छे कलाकार होते है। सर्कस में तो जानवर भी काम // करते है। वहाँ हाथी, श